<a href="https://colab.research.google.com/github/ANDIsign/especializacion/blob/main/Modelo_final_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de dependencias
!pip install google-play-scraper transformers torch

# Importación de librerías
import torch
import pandas as pd
import numpy as np
import re
from google_play_scraper import Sort, reviews
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample


# Configuración inicial
CONFIG = {
    'app_id': 'com.netflix.mediaclient',
    'lang': 'es',
    'country': 'co',
    'sort': Sort.NEWEST,
    'max_len': 30,
    'batch_size': 32,
    'epochs': 10,
    'n_classes': 2,
    'random_seed': 42,
    'pre_trained_model_name': 'bert-base-cased'
}

# Configuración de semilla para reproducibilidad
np.random.seed(CONFIG['random_seed'])
torch.manual_seed(CONFIG['random_seed'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo usado: {device}")

# Función para limpiar el texto
def limpiar_texto(texto):
    if isinstance(texto, str):
        texto = texto.lower()
        texto = re.sub(r"http\S+|www\S+|https\S+", '', texto)
        texto = re.sub(r'\@\w+|\#', '', texto)
        texto = re.sub(r"[^a-zA-Z0-9áéíóúñÁÉÍÓÚÑ\s]", '', texto)
    return texto

# Función para preparar el dataset y etiquetar las reseñas
def prepare_dataset(df, label_threshold=3):
    df['label'] = df['score'].apply(lambda x: 0 if x <= label_threshold else 1)
    return df[['content', 'label']].dropna().reset_index(drop=True)


# Función para extraer reseñas
def obtener_reseñas(app_id, lang, country, sort):
    all_reviews = []
    next_token = None
    while True:
        reviews_batch, next_token = reviews(
            app_id,
            lang=lang,
            country=country,
            sort=sort,
            continuation_token=next_token
        )
        # Filtrar las reseñas del año 2024
        reviews_filtered = [r for r in reviews_batch if r['at'].year == 2024]
        if not reviews_filtered:
            break
        all_reviews.extend(reviews_filtered)

        if not next_token or not reviews_filtered:
            break
    return pd.DataFrame(all_reviews)

# Obtener reseñas de la aplicación
df = obtener_reseñas(CONFIG['app_id'], CONFIG['lang'], CONFIG['country'], CONFIG['sort'])

# Verificar si se obtuvieron datos
if df.empty:
    raise ValueError("No se obtuvieron datos de la API. Verifica la configuración.")

# Limpiar el DataFrame
df['content'] = df['content'].apply(limpiar_texto)

# Preparar el dataset
df_modelo = prepare_dataset(df)

print(f"Número de reseñas extraídas: {len(df_modelo)}")

# Tomar una muestra aleatoria de n ejemplos
df_modelo_sample = df_modelo.sample(n=1000, random_state=CONFIG['random_seed']).reset_index(drop=True)

# Balancear el dataset
def balancear_datos(df):
    clase_mayoritaria = df[df.label == df.label.value_counts().idxmax()]
    clase_minoritaria = df[df.label != df.label.value_counts().idxmax()]
    clase_minoritaria_oversampled = resample(
        clase_minoritaria,
        replace=True,
        n_samples=len(clase_mayoritaria),
        random_state=CONFIG['random_seed']
    )
    df_balanceado = pd.concat([clase_mayoritaria, clase_minoritaria_oversampled]).sample(frac=1, random_state=CONFIG['random_seed'])
    return df_balanceado

# Balancear la muestra
df_modelo_sample_balanceado = balancear_datos(df_modelo_sample)

# Dividir el dataset balanceado en entrenamiento y prueba
df_train, df_test = train_test_split(df_modelo_sample_balanceado, test_size=0.2, random_state=CONFIG['random_seed'])

# Dataset personalizado para BERT
class SentimentDataset(Dataset):
    def __init__(self, contents, labels, tokenizer, max_len):
        self.contents = contents
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.contents)

    def __getitem__(self, idx):
        content = str(self.contents[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            content,
            max_length=self.max_len,
            truncation=True,
            add_special_tokens=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Función para crear el DataLoader
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = SentimentDataset(
        contents=df.content.to_numpy(),
        labels=df.label.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(dataset, batch_size=batch_size, num_workers=2)

# Crear DataLoaders
tokenizer = BertTokenizer.from_pretrained(CONFIG['pre_trained_model_name'])
train_data_loader = create_data_loader(df_train, tokenizer, CONFIG['max_len'], CONFIG['batch_size'])
test_data_loader = create_data_loader(df_test, tokenizer, CONFIG['max_len'], CONFIG['batch_size'])

# Modelo BERT
class BERTSentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BERTSentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(CONFIG['pre_trained_model_name'])
        self.drop = nn.Dropout(p=0.2)
        self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        cls_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        return self.linear(self.drop(cls_output))

# Configuración del modelo
model = BERTSentimentClassifier(CONFIG['n_classes']).to(device)
optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01) # lr=1e-6 anterior
total_steps = len(train_data_loader) * CONFIG['epochs']
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss().to(device)

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0, verbose=False, path='best_model.pt'):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.verbose = verbose
        self.path = path

    def __call__(self, val_loss, model):
        if self.best_loss is None or val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.save_checkpoint(model)
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping contador: {self.counter} de {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

    def save_checkpoint(self, model):
        if self.verbose:
            print('Guardando el mejor modelo ...')
        torch.save(model.state_dict(), self.path)

# Funciones de entrenamiento y evaluación
def train_epoch(model, data_loader, loss_fn, optimizer, scheduler, device, n_examples):
    model.train()
    correct_predictions, losses = 0, []
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, labels)
        _, preds = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model.eval()
    correct_predictions, losses = 0, []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs, labels)
            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
    return correct_predictions.double() / n_examples, np.mean(losses)

# Entrenamiento y evaluación
def train_model(model, train_data_loader, test_data_loader, optimizer, scheduler, loss_fn, n_train_examples, n_test_examples):
    best_accuracy = 0
    early_stopping = EarlyStopping(patience=3, verbose=True)
    for epoch in range(CONFIG['epochs']):
        print(f"Epoch {epoch + 1}/{CONFIG['epochs']}")

        # Entrenamiento
        train_acc, train_loss = train_epoch(model, train_data_loader, loss_fn, optimizer, scheduler, device, n_train_examples)
        print(f"Entrenamiento - Precisión: {train_acc} - Pérdida: {train_loss}")

        # Evaluación
        val_acc, val_loss = eval_model(model, test_data_loader, loss_fn, device, n_test_examples)
        print(f"Evaluación - Precisión: {val_acc} - Pérdida: {val_loss}")

        # Early Stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Entrenamiento detenido temprano")
            break

        # Guardar el mejor modelo
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            torch.save(model.state_dict(), 'best_model.pt')

# Entrenar el modelo
train_model(model, train_data_loader, test_data_loader, optimizer, scheduler, loss_fn, len(df_train), len(df_test))

# Cargar el mejor modelo
model.load_state_dict(torch.load('best_model.pt'))

# Evaluación final
y_pred = []
y_true = df_test['label'].tolist()
for batch in test_data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        y_pred.extend(preds.cpu().numpy())

# Reporte de métricas
print("Reporte de clasificación:\n", classification_report(y_true, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular la matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Visualizar la matriz de confusión con un mapa de calor
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Clase 0', 'Clase 1'], yticklabels=['Clase 0', 'Clase 1'])
plt.xlabel('Predicción')
plt.ylabel('Verdadero')
plt.title('Matriz de Confusión')
plt.show()

# Cálculos de métricas
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Imprimir las métricas
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


In [ ]:
from sklearn.metrics import roc_curve, auc

# Calcular probabilidades de clase positiva
y_probs = []  # Probabilidades de predicción de la clase positiva
model.eval()
for batch in test_data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.nn.functional.softmax(outputs, dim=1)[:, 1]
        y_probs.extend(probs.cpu().numpy())

# Calcular curva ROC
fpr, tpr, thresholds = roc_curve(y_true, y_probs)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Línea aleatoria
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

# Calcular precisión y recall en diferentes umbrales
precision, recall, thresholds = precision_recall_curve(y_true, y_probs)
average_precision = average_precision_score(y_true, y_probs)

# Graficar la curva de Precisión-Recall
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='green', lw=2, label=f'AP = {average_precision:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precisión')
plt.title('Curva Precisión-Recall')
plt.legend(loc="lower left")
plt.grid(True)
plt.show()


In [ ]:
# Distribución de probabilidades predichas
plt.figure(figsize=(8, 6))
plt.hist(y_probs, bins=20, color='blue', alpha=0.7, label='Probabilidad Clase 1')
plt.axvline(x=0.5, color='red', linestyle='--', label='Umbral 0.5')
plt.xlabel('Probabilidad Predicha')
plt.ylabel('Frecuencia')
plt.title('Distribución de Probabilidades de Predicción')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Extraer Falsos Positivos y Falsos Negativos
df_test['y_pred'] = y_pred
falsos_positivos = df_test[(df_test['label'] == 0) & (df_test['y_pred'] == 1)]
falsos_negativos = df_test[(df_test['label'] == 1) & (df_test['y_pred'] == 0)]

print(f"Número de Falsos Positivos: {len(falsos_positivos)}")
print(f"Número de Falsos Negativos: {len(falsos_negativos)}")

# Opcional: Visualizar ejemplos específicos
print("Ejemplos de Falsos Positivos:")
print(falsos_positivos['content'].head())

print("Ejemplos de Falsos Negativos:")
print(falsos_negativos['content'].head())


In [ ]:
# Encontrar el umbral óptimo
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold = thresholds[np.argmax(f1_scores)]
print(f"Umbral óptimo: {best_threshold}")


In [ ]:
from sklearn.metrics import classification_report

# Recalcular predicciones con el umbral óptimo
optimal_threshold = 0.058853041380643845
y_pred_optimal = [1 if prob > optimal_threshold else 0 for prob in y_probs]

# Generar el reporte de clasificación
print("Reporte de clasificación con umbral óptimo:")
print(classification_report(y_true, y_pred_optimal))


In [ ]:
# Graficar la Curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')

# Ajustar el índice para el umbral óptimo
threshold_idx = np.argmin(np.abs(thresholds - optimal_threshold))
if threshold_idx >= len(fpr):  # Garantizar que el índice esté dentro del rango
    threshold_idx = len(fpr) - 1

# Agregar el punto del umbral óptimo a la curva
plt.scatter(fpr[threshold_idx],
            tpr[threshold_idx],
            color='red', label=f'Umbral Óptimo ({optimal_threshold:.2f})')

# Línea diagonal aleatoria
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC con Umbral Óptimo')
plt.legend(loc="lower right")
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Recalcular predicciones con el umbral óptimo
y_pred_optimal = [1 if prob > optimal_threshold else 0 for prob in y_probs]

# Generar el reporte de clasificación
print("Reporte de clasificación con el umbral óptimo:")
print(classification_report(y_true, y_pred_optimal))


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular predicciones con umbral por defecto (0.5)
y_pred_default = [1 if prob > 0.5 else 0 for prob in y_probs]

# Calcular predicciones con umbral óptimo
optimal_threshold = 0.08675912022590637
y_pred_optimal = [1 if prob > optimal_threshold else 0 for prob in y_probs]

# Generar matrices de confusión
cm_default = confusion_matrix(y_true, y_pred_default)
cm_optimal = confusion_matrix(y_true, y_pred_optimal)

# Función para visualizar la matriz de confusión
def plot_confusion_matrix(cm, title, labels=['Clase 0', 'Clase 1']):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.xlabel('Predicción')
    plt.ylabel('Verdadero')
    plt.show()

# Visualizar matriz con umbral por defecto
plot_confusion_matrix(cm_default, 'Matriz de Confusión - Umbral por Defecto (0.5)')

# Visualizar matriz con umbral óptimo
plot_confusion_matrix(cm_optimal, f'Matriz de Confusión - Umbral Óptimo ({optimal_threshold:.4f})')

# Generar reporte de clasificación con ambos umbrales
print("Reporte de Clasificación - Umbral por Defecto (0.5):")
print(classification_report(y_true, y_pred_default))

print("\nReporte de Clasificación - Umbral Óptimo:")
print(classification_report(y_true, y_pred_optimal))


In [ ]:
# Asegúrate de que y_pred_optimal esté calculado y agregado al DataFrame
df_test['y_pred_optimal'] = [1 if prob > optimal_threshold else 0 for prob in y_probs]

# Identificar los falsos positivos y falsos negativos
falsos_positivos = df_test[(df_test['label'] == 0) & (df_test['y_pred_optimal'] == 1)]
falsos_negativos = df_test[(df_test['label'] == 1) & (df_test['y_pred_optimal'] == 0)]

# Mostrar ejemplos
print("Falsos Positivos (Clase 0 predicha como 1):")
print(falsos_positivos['content'].head())

print("\nFalsos Negativos (Clase 1 predicha como 0):")
print(falsos_negativos['content'].head())


In [ ]:
# Contar promotores y detractores
promotores = sum(1 for pred in y_pred if pred == 1)  # Comentarios positivos (Promotores)
detractores = sum(1 for pred in y_pred if pred == 0)  # Comentarios negativos (Detractores)

# Calcular el total de comentarios
total_comentarios = len(y_pred)

# Calcular el NPS
nps = ((promotores - detractores) / total_comentarios) * 100

# Imprimir los resultados
print(f"Promotores (Positivos): {promotores}")
print(f"Detractores (Negativos): {detractores}")
print(f"Total de Comentarios: {total_comentarios}")
print(f"NPS: {nps:.2f}")


In [ ]:
'''
Aplicamos el modelo a todo el dataset con el fin de identificar el total de comentarios positivos y negativos, así calcularemos el NPS homologando Detractor = Comentario Negativo, Promotor = Comentario Positivo.
Luego se hará una comparación vs el NPS calculado desde el lado de las calificaciones o número de estrellas
'''

In [ ]:
# Crear un DataLoader para todos los comentarios
class AllCommentsDataset(Dataset):
    def __init__(self, contents, tokenizer, max_len):
        self.contents = contents
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.contents)

    def __getitem__(self, idx):
        content = str(self.contents[idx])
        encoding = self.tokenizer.encode_plus(
            content,
            max_length=self.max_len,
            truncation=True,
            add_special_tokens=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# Preparar el DataLoader para todos los comentarios
all_comments_dataset = AllCommentsDataset(
    contents=df['content'].to_numpy(),
    tokenizer=tokenizer,
    max_len=CONFIG['max_len']
)
all_comments_loader = DataLoader(all_comments_dataset, batch_size=CONFIG['batch_size'], num_workers=2)

# Hacer predicciones para todos los comentarios
all_preds = []
model.eval()
with torch.no_grad():
    for batch in all_comments_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())

# Agregar las predicciones al DataFrame original
df['predicted_label'] = all_preds

# Calcular el NPS
promotores = df[df['predicted_label'] == 1].shape[0]  # Comentarios positivos
detractores = df[df['predicted_label'] == 0].shape[0]  # Comentarios negativos
total_comentarios = len(df)

nps_total = ((promotores - detractores) / total_comentarios) * 100

# Imprimir los resultados
print(f"Total de comentarios: {total_comentarios}")
print(f"Promotores (Positivos): {promotores}")
print(f"Detractores (Negativos): {detractores}")
print(f"NPS Total: {nps_total:.2f}")


In [ ]:
# 1. Calcular el NPS real basado en las calificaciones originales
df['real_label'] = df['score'].apply(lambda x: 0 if x <= 3 else (1 if x == 5 else None))  # Neutros excluidos

# Contar detractores y promotores reales
detractores_reales = df[df['real_label'] == 0].shape[0]
promotores_reales = df[df['real_label'] == 1].shape[0]
total_comentarios_reales = detractores_reales + promotores_reales  # Excluimos neutros

# Calcular NPS real
nps_real = ((promotores_reales - detractores_reales) / total_comentarios_reales) * 100

# 2. Calcular el NPS predicho basado en el modelo
promotores_predichos = df[df['predicted_label'] == 1].shape[0]
detractores_predichos = df[df['predicted_label'] == 0].shape[0]
total_comentarios_predichos = len(df)

# Calcular NPS predicho
nps_predicho = ((promotores_predichos - detractores_predichos) / total_comentarios_predichos) * 100

# 3. Comparar ambos NPS
print(f"Resultados de NPS:")
print(f"NPS Real (Basado en calificaciones originales): {nps_real:.2f}")
print(f"NPS Predicho (Basado en modelo): {nps_predicho:.2f}")

# Opcional: Comparar distribución de comentarios reales y predichos
print("\nDistribución de comentarios:")
print(f"Detractores reales: {detractores_reales}, Promotores reales: {promotores_reales}")
print(f"Detractores predichos: {detractores_predichos}, Promotores predichos: {promotores_predichos}")


In [ ]:
# Filtrar comentarios negativos
comentarios_negativos = df[df['predicted_label'] == 0]['content']
print(f"Total de comentarios negativos: {len(comentarios_negativos)}")


In [ ]:
!pip install scikit-learn sentence-transformers spacy pandas
!python -m spacy download es_core_news_sm

In [ ]:
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import spacy
import pandas as pd
from collections import Counter

# Cargar modelo de idioma en español
nlp = spacy.load("es_core_news_sm")

# Modelo para embeddings de oraciones
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Función para extraer frases completas con contexto
def extraer_frases_completas(texto):
    """
    Extrae frases contextualmente completas del texto.
    """
    doc = nlp(texto)
    frases = []
    for sent in doc.sents:  # Iterar sobre oraciones completas
        sustantivos = [token for token in sent if token.pos_ == "NOUN"]
        if sustantivos:
            # Construir frases con sustantivos y modificadores relevantes
            for noun in sustantivos:
                modificadores = [child.text for child in noun.children if child.pos_ in {"ADJ", "VERB", "ADV"}]
                frase = f"{noun.text} {' '.join(modificadores)}"
                frases.append(frase.strip())
    return frases

# Extraer comentarios negativos del DataFrame
comentarios_negativos = df[df['predicted_label'] == 0]['content'].dropna().tolist()

# Preprocesar los comentarios negativos para extraer frases completas
comentarios_frases = []
for comentario in comentarios_negativos:
    frases = extraer_frases_completas(comentario)
    comentarios_frases.extend(frases)

# Validar que no estén vacías
if not comentarios_frases:
    raise ValueError("No hay frases clave válidas después del procesamiento.")

# Obtener embeddings semánticos para frases completas
embeddings = embedder.encode(comentarios_frases)

# Agrupar frases similares usando K-Means
num_clusters = 5  # Número de temas que queremos encontrar
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
kmeans.fit(embeddings)

# Asociar frases a sus respectivos clusters
comentarios_cluster = pd.DataFrame({
    'frase': comentarios_frases,
    'cluster': kmeans.labels_
})

# Función para seleccionar frases representativas completas
def obtener_frases_representativas_completas(cluster_df, num_frases=5):
    """
    Selecciona frases completas más representativas para cada cluster.
    """
    frases_por_cluster = {}
    for cluster in sorted(cluster_df['cluster'].unique()):
        frases = cluster_df[cluster_df['cluster'] == cluster]['frase'].tolist()
        if not frases:
            continue  # Saltar clusters vacíos
        # Seleccionar las frases más frecuentes
        frases_frecuentes = [frase for frase, _ in Counter(frases).most_common(num_frases)]
        frases_por_cluster[f"Tema {cluster + 1}"] = frases_frecuentes
    return frases_por_cluster

# Obtener frases representativas para cada cluster
temas = obtener_frases_representativas_completas(comentarios_cluster)

# Mostrar resultados
print("\nTemas principales de comentarios negativos (con frases completas):")
for tema, frases in temas.items():
    print(f"{tema}: {', '.join(frases)}")
